# Graph Deep Learning on Graph to study Fake News

Our dataset is https://arxiv.org/pdf/2104.12259 .
The one in the article to study create two kind of nodes: one per authors, one per tweet. But here there is only a node per authors, not per tweets. Hence it is more focus on the users-spreading. The datas we store for an author are the same as the article have to study except that we lack social networks part, in particular:
- BERT is a word-embedding of the preferences of the author (the result of a transformer fed with its average tweets)
- Profile is a vector representing the author informations (number of followers, like, etc.)

In [156]:
!pip install dgl

In [157]:
!pip install torch_geometric

In [5]:
from torch_geometric.datasets import UPFD
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from torch.nn.modules import Linear
from torch_geometric.transforms import ToUndirected
from torch.functional import F
import torch
import argparse
import os.path as osp


The dataset is initialized there. Note that:
- `feature` can be `content` (raw content of the tweet), `bert` (content transformed by a transformer), `profile` (user profile info such as number of tweets, followers, and join date), `spacy` (content transformed by a simple NLP model)
- `dataset` is either `politifact` or `gossipcop`

In [328]:
from torch_geometric.data import DataLoader 
from torch_geometric.datasets import UPFD
import os.path as osp
import torch

# Paths and settings
_file_ = '..'
file = _file_
dataset = 'gossipcop'  # or 'politifact'
path = osp.join(osp.dirname(osp.realpath(file)), '..', 'data', 'UPFD')

# Load datasets with bert and profile features separately
train_dataset_bert = UPFD(path, dataset, 'bert', 'train')
train_dataset_profile = UPFD(path, dataset, 'profile', 'train')

val_dataset_bert = UPFD(path, dataset, 'bert', 'val')
val_dataset_profile = UPFD(path, dataset, 'profile', 'val')

test_dataset_bert = UPFD(path, dataset, 'bert', 'test')
test_dataset_profile = UPFD(path, dataset, 'profile', 'test')

# Check that both datasets are aligned
assert len(train_dataset_bert) == len(train_dataset_profile)
assert len(val_dataset_bert) == len(val_dataset_profile)
assert len(test_dataset_bert) == len(test_dataset_profile)

# Function to combine features
def combine_features(dataset_bert, dataset_profile):
    combined_data = []
    for data_bert, data_profile in zip(dataset_bert, dataset_profile):
        data_bert.x = torch.cat([data_bert.x, data_profile.x], dim=-1)  # Concatenate features
        combined_data.append(data_bert)
    return combined_data

# Combine features for train, val, and test datasets
train_dataset = combine_features(train_dataset_bert, train_dataset_profile)
val_dataset = combine_features(val_dataset_bert, val_dataset_profile)
test_dataset = combine_features(test_dataset_bert, test_dataset_profile)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print(len(train_dataset) + len(val_dataset) + len(test_dataset))

5464


In [329]:
print(train_dataset[0].x.shape)
print(sum([e.x.shape[0] for e in train_dataset[0:128] ]))
for e in train_loader:
    s = [0 for i in range(128)]
    for k in e.batch:
        s[k]+=1
    print(s)
    break

torch.Size([76, 778])
6883
[79, 70, 92, 52, 45, 55, 133, 46, 61, 193, 7, 60, 91, 17, 23, 151, 44, 115, 133, 44, 137, 10, 23, 189, 73, 10, 165, 58, 51, 28, 12, 16, 80, 66, 56, 43, 163, 51, 164, 30, 183, 8, 27, 82, 79, 104, 101, 22, 77, 8, 16, 110, 42, 67, 139, 26, 4, 98, 110, 85, 66, 117, 11, 26, 33, 30, 9, 18, 10, 17, 18, 16, 49, 22, 31, 35, 25, 49, 4, 45, 23, 14, 8, 167, 88, 27, 79, 77, 10, 19, 16, 51, 50, 76, 8, 78, 95, 6, 20, 5, 4, 160, 184, 6, 19, 111, 54, 32, 113, 20, 10, 17, 142, 100, 144, 18, 36, 76, 14, 75, 40, 21, 125, 59, 88, 7, 104, 82]


In [330]:
print(train_dataset)
print(train_dataset[0].y)
print(train_dataset[0].x)
print(train_dataset[0].edge_index)
#train_dataset[i] représente l'arbre i
#Le champ x représente la valeur des noeud, il est de taille (n, 778).
#Le champ edge index représente les connexion dans l'abre. Il est de taille (2, n - 1). edge_index[0][i] est
#le père de edge_index[1][i]. En pratique, il se trouve que edge_index[1][i] est toujours un linespace, ce qui est possible
#Le champ y représente si c'est un graphe de fake news ou pas

[Data(x=[76, 778], edge_index=[2, 75], y=[1]), Data(x=[125, 778], edge_index=[2, 124], y=[1]), Data(x=[6, 778], edge_index=[2, 5], y=[1]), Data(x=[56, 778], edge_index=[2, 55], y=[1]), Data(x=[90, 778], edge_index=[2, 89], y=[1]), Data(x=[21, 778], edge_index=[2, 20], y=[1]), Data(x=[76, 778], edge_index=[2, 75], y=[1]), Data(x=[89, 778], edge_index=[2, 88], y=[1]), Data(x=[77, 778], edge_index=[2, 76], y=[1]), Data(x=[27, 778], edge_index=[2, 26], y=[1]), Data(x=[88, 778], edge_index=[2, 87], y=[1]), Data(x=[127, 778], edge_index=[2, 126], y=[1]), Data(x=[88, 778], edge_index=[2, 87], y=[1]), Data(x=[19, 778], edge_index=[2, 18], y=[1]), Data(x=[173, 778], edge_index=[2, 172], y=[1]), Data(x=[54, 778], edge_index=[2, 53], y=[1]), Data(x=[19, 778], edge_index=[2, 18], y=[1]), Data(x=[88, 778], edge_index=[2, 87], y=[1]), Data(x=[24, 778], edge_index=[2, 23], y=[1]), Data(x=[72, 778], edge_index=[2, 71], y=[1]), Data(x=[12, 778], edge_index=[2, 11], y=[1]), Data(x=[38, 778], edge_index=

### Model

In [3]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 concat=False):
        super().__init__()
        self.concat = concat
        self.conv1 = GATConv(in_channels, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.max_pooling = global_mean_pool # They use Max_pool in the article but that doesn't work pretty well.
        self.lin1 = Linear(hidden_channels, 2 * hidden_channels)
        self.lin2 = Linear(2*hidden_channels,2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.selu(x)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.selu(x)
        x = self.max_pooling(x, batch)
        x = torch.nn.functional.selu(x)
        x = self.lin1(x)
        x = torch.nn.functional.selu(x)
        x = self.lin2(x)
        return x.softmax(dim=-1)

In [332]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(778, 256,2, concat=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.01)

In [333]:
loss_fn = torch.nn.HingeEmbeddingLoss()
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        y = torch.tensor([[1,0] if e==1 else [0,1] for e in data.y]).to(device)
        loss = loss_fn(out,y)
        #loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_loader.dataset)
train()

0.745829510601449

In [8]:
@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = total_examples = 0
    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.edge_index, data.batch).argmax(dim=-1)
        total_correct += int((pred ==
         data.y).sum())
        total_examples += data.num_graphs

    return total_correct / total_examples

In [335]:
for epoch in range(1, 200):
    loss = train()
    train_acc = test(train_loader)
    val_acc = test(val_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, 'f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 01, Loss: 0.7442, Train: 0.4899, Val: 0.5147, Test: 0.5010
Epoch: 02, Loss: 0.7334, Train: 0.5101, Val: 0.4853, Test: 0.4992
Epoch: 03, Loss: 0.7438, Train: 0.7115, Val: 0.6996, Test: 0.6952
Epoch: 04, Loss: 0.7432, Train: 0.5366, Val: 0.5128, Test: 0.5254
Epoch: 05, Loss: 0.7396, Train: 0.5769, Val: 0.5879, Test: 0.5719
Epoch: 06, Loss: 0.6868, Train: 0.7234, Val: 0.7308, Test: 0.7125
Epoch: 07, Loss: 0.6398, Train: 0.7848, Val: 0.7839, Test: 0.7574
Epoch: 08, Loss: 0.6337, Train: 0.6438, Val: 0.6410, Test: 0.6281
Epoch: 09, Loss: 0.6531, Train: 0.7308, Val: 0.7088, Test: 0.7033
Epoch: 10, Loss: 0.6187, Train: 0.7701, Val: 0.7766, Test: 0.7486
Epoch: 11, Loss: 0.6935, Train: 0.5101, Val: 0.4853, Test: 0.4992
Epoch: 12, Loss: 0.7111, Train: 0.6612, Val: 0.6685, Test: 0.6597
Epoch: 13, Loss: 0.6353, Train: 0.8251, Val: 0.8278, Test: 0.7899
Epoch: 14, Loss: 0.6104, Train: 0.7051, Val: 0.7308, Test: 0.7015
Epoch: 15, Loss: 0.6384, Train: 0.7802, Val: 0.7674, Test: 0.7656
Epoch: 16,

We have the same accuracy, we need to do the ROC to be sure.